In [1]:
import glob
import numpy as np
import lasio
import pandas as pd
import json
from pandas import merge_ordered

## Define helper functions

In [48]:
def load_data(filename):
    l = lasio.read(filename)
    data_well = l.df()
    log = list(data_well.columns.values)
    wellname=filename.split('\\')
    wellname=wellname[-1]
    header = [{
        'Well Name': wellname,
        'START':l.well.STRT.value,
        'STOP':l.well.STOP.value,
        'STEP':l.well.STEP.value,
        'Location': filename
    }]
    data_well['WELL'] = filename  
    log_list = pd.DataFrame(header)
    return data_well, log_list

def merge_alias(db, alias, logs_selected):
    well = db['WELL'].unique()
    merged_data = pd.DataFrame()
    for i in range(len(well)):
        data = db.where(db['WELL']==well[i]).dropna(axis=1, how='all')
        for j in range(len(alias)):
            welllog_name = list(set(data.columns).intersection(alias.get(list(alias)[j])))
            samelog = data[welllog_name]
            count_log = dict(sorted(zip(welllog_name, samelog.count()), key=lambda item: item[1], reverse=True))
            welllog_name = list(count_log.keys())
            if (len(welllog_name)!=0):
                #If more than one log aliases exist, normalize each log to have same data range in the same depth
                if (len(welllog_name)>1):
                    alias_logs = data[welllog_name].dropna()
                    if (len(alias_logs)!=0):
                        a = []; b = []; c = []
                        for n in range(len(alias_logs.columns)):
                            q1 = alias_logs[welllog_name[n]].quantile(0.1)
                            q9 = alias_logs[welllog_name[n]].quantile(0.9)
                            a.append(q1)
                            b.append(q9)
                            c = [b-a for (a,b) in zip(a,b)]
                            c = list(map(lambda x: x/c[0],c))
                        for n in range(len(welllog_name)):
                            data.loc[:, welllog_name[n]] *= 1/c[n]
                    for k in range(len(welllog_name)-1):
                        data[welllog_name[0]].fillna(data[welllog_name[k+1]], inplace=True)
                data[list(alias)[j]] = data[welllog_name[0]]   
        merged_data = merged_data.append(data)
        merged_data = merged_data[merged_data['WELL'].notna()]
        new_list=[]
        for x in logs_selected:
            if x in list(merged_data.columns):
                new_list.append(x)
        merged_data = merged_data[new_list]
    return merged_data, data

def tablecheck(filename, logs_selected):
    data_well, log_list=load_data(filename)
    merged_data, data=merge_alias(data_well, alias, logs_selected)
    df=pd.DataFrame([["X"]*(len(list(merged_data.columns)))], columns=list(merged_data.columns))
    df2=pd.concat([df, log_list], axis=1)
    return df2, log_list

## Open files

In [59]:
with open('alias.json') as file:
    alias = json.load(file)
logs_selected= ['CAL', 'RXO', 'GR', 'POR', 'DRES', 'DT', 'DENS', 'DRHO']

df=pd.DataFrame(columns=logs_selected)
for filename in glob.iglob(r'C:\Users\Henky\JUPYTER\edit las\**\*.las', recursive=True):
    df2, log_list=tablecheck(filename, logs_selected)
    df=df.append(df2, ignore_index=True, sort=False)
header=(list(log_list.columns) + logs_selected)
df=df[header]
print (df)
df.to_excel("log checklist.xlsx")  

            Well Name      START         STOP   STEP  \
0          well-1.las  3001.0000  4460.000000  0.500   
1          well-2.las  3001.0000  4521.500000  0.500   
2          well-3.las  3001.0000  4759.500000  0.500   
3         15_9-15.las    25.0000  3210.005615  0.152   
4            puk1.las  1860.0000  4918.000000  0.500   
5  Yvonne_04_logs.las   506.2728  1452.981600  0.000   

                                            Location CAL RXO GR POR DRES   DT  \
0         C:\Users\Henky\JUPYTER\edit las\well-1.las   X   X  X   X    X    X   
1         C:\Users\Henky\JUPYTER\edit las\well-2.las   X   X  X   X    X    X   
2         C:\Users\Henky\JUPYTER\edit las\well-3.las   X   X  X   X    X    X   
3    C:\Users\Henky\JUPYTER\edit las\new\15_9-15.las   X   X  X   X  NaN  NaN   
4       C:\Users\Henky\JUPYTER\edit las\new\puk1.las   X   X  X   X    X    X   
5  C:\Users\Henky\JUPYTER\edit las\new\Yvonne_04_...   X   X  X   X    X    X   

  DENS DRHO  
0    X  NaN  
1    X  NaN

In [57]:
def logCheck(filepath, log_selected):
    with open('alias.json') as file:
        alias = json.load(file) #load the JSON File
    df=pd.DataFrame(columns=logs_selected)
    for filename in glob.iglob(filepath, recursive=True):
        wellname=filename.split('\\')
        wellname=wellname[-1]
        df['Well Name']=wellname
        df2, log_list=tablecheck(filename, logs_selected)
        df=df.append(df2, ignore_index=True, sort=False)
    header=(list(log_list.columns) + logs_selected)
    df=df[header]
    return df

In [58]:
#define the logs that we are interested in our las files, select based on the alias name on the JSON file
logs_selected= ['CAL', 'RXO', 'GR', 'POR', 'DRES', 'DT', 'DENS', 'DRHO'] 
filepath=r'D:\Work\HPI\Clients files\HOPE(OSES)\**\*.las'

checkreport= logCheck(filepath, logs_selected)
print (checkreport)

              Well Name      START       STOP  STEP  \
0   Zelda_E-10_logs.las   506.2728  1452.9816   0.0   
1   Zelda_E-10_logs.las   244.4496  3446.0688   0.0   
2   Zelda_E-10_logs.las   702.8688  2670.6577   0.0   
3   Zelda_E-10_logs.las  1446.5808  2618.8416   0.0   
4   Zelda_E-10_logs.las   556.2600  2313.1272   0.0   
5   Zelda_E-10_logs.las   639.1656  2030.2728   0.0   
6   Zelda_E-10_logs.las   641.2992  2201.2656   0.0   
7   Zelda_E-10_logs.las    32.3088  3379.8420   0.0   
8   Zelda_E-10_logs.las   238.6584  1296.6193   0.0   
9   Zelda_E-10_logs.las   602.8944  1380.4392   0.0   
10  Zelda_E-10_logs.las   615.0864  1586.7888   0.0   
11  Zelda_E-10_logs.las   592.6836  1607.2104   0.0   
12  Zelda_E-10_logs.las   298.0944  1477.6704   0.0   
13  Zelda_E-10_logs.las  1515.1608  2301.5449   0.0   
14  Zelda_E-10_logs.las   300.8376  1419.1488   0.0   
15  Zelda_E-10_logs.las   297.4848  1483.4617   0.0   
16  Zelda_E-10_logs.las   706.8312  2243.0232   0.0   
17  Zelda_